# Content based filtering:

## Recommend based on user's genre preferences.

Import needed libraries

In [1]:
import pandas as pd
import numpy as np
import math
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



### Load the movies dataset 

In [3]:
Path_to_drive = '/content/drive/MyDrive/Recommender_Sys/'
csv_file_name = Path_to_drive + 'movies.csv'
Movies_df = pd.read_csv(csv_file_name)
print("Movies_df Shape : {}\n\n".format(Movies_df.shape))
Movies_df.head()

Movies_df Shape : (9742, 3)




,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### Time to explore the movies and clean them

Remove the year from the titile column and put it in a column on it's own calling it year

In [4]:
Movies_df['year'] = Movies_df.title.str.findall('\(\d\d\d\d\)')
Movies_df['year']

0       [(1995)]
1       [(1995)]
2       [(1995)]
3       [(1995)]
4       [(1995)]
          ...   
9737    [(2017)]
9738    [(2017)]
9739    [(2017)]
9740    [(2018)]
9741    [(1991)]
Name: year, Length: 9742, dtype: object

In [5]:
Movies_df['year'] = Movies_df['year'].astype(str)
Movies_df['year'] = Movies_df.year.str.replace('\[\'\(','', regex=True)
Movies_df['year'] = Movies_df.year.str.replace('\)\'\]','', regex=True)
Movies_df

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017
9739,193585,Flint (2017),Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018


In [6]:
Movies_df.title=Movies_df.title.str.replace('\(\d\d\d\d\)','', regex=True)
Movies_df['title'] = Movies_df['title'].apply(lambda x: x.strip())
Movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Now kindly transform the genre column into a list 

In [7]:
Movies_df['genres'] = Movies_df['genres'].str.split('|', expand=False)
Movies_df['genres']

0       [Adventure, Animation, Children, Comedy, Fantasy]
1                          [Adventure, Children, Fantasy]
2                                       [Comedy, Romance]
3                                [Comedy, Drama, Romance]
4                                                [Comedy]
                              ...                        
9737                 [Action, Animation, Comedy, Fantasy]
9738                         [Animation, Comedy, Fantasy]
9739                                              [Drama]
9740                                  [Action, Animation]
9741                                             [Comedy]
Name: genres, Length: 9742, dtype: object

Now view a summary of the data

In [8]:
Movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
 3   year     9742 non-null   object
dtypes: int64(1), object(3)
memory usage: 304.6+ KB


Check for missing values

In [9]:
Movies_df.isnull().sum()

movieId    0
title      0
genres     0
year       0
dtype: int64

Fill movies_df missing year  values with 0 to indicate the year is not available.

In [10]:
Movies_df['year'] = Movies_df['year'].str.replace('\]','').str.replace('\[','')

In [11]:
# replace field that's entirely space (or empty) with 0
Movies_df = Movies_df.replace(r'^\s*$', '0', regex=True)

In [12]:
# convert year column from int6a to int8, since it holds a max of just 4 digits of numbers
Movies_df['year'] = Movies_df['year'].astype('int16')
Movies_df['title'] = Movies_df['title'].astype('str')

Movies_df['title'].dtype

dtype('O')

In [13]:
Movies_df

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]",2017
9738,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]",2017
9739,193585,Flint,[Drama],2017
9740,193587,Bungo Stray Dogs: Dead Apple,"[Action, Animation]",2018


In [14]:
Movies_df['genres']

0       [Adventure, Animation, Children, Comedy, Fantasy]
1                          [Adventure, Children, Fantasy]
2                                       [Comedy, Romance]
3                                [Comedy, Drama, Romance]
4                                                [Comedy]
                              ...                        
9737                 [Action, Animation, Comedy, Fantasy]
9738                         [Animation, Comedy, Fantasy]
9739                                              [Drama]
9740                                  [Action, Animation]
9741                                             [Comedy]
Name: genres, Length: 9742, dtype: object

One-Hot-Encode the list of genres


In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(sparse_output=True)

Movies_df = Movies_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(Movies_df.pop('genres')),
                index=Movies_df.index,
                columns=mlb.classes_))

In [16]:
Movies_df.columns

Index(['movieId', 'title', 'year', '(no genres listed)', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [17]:
Movies_df.head()

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# If your OHE includes nans fill in the NaN values with 0
Movies_df = Movies_df.replace(np.nan, 0)

## Building the Content Based recommender System

Assuming that our user called Lawrence, let's create Lawrence's Profile

In [19]:
# so on a scale of 0 to 5, with 0 min and 5 max, see Lawrence's movie ratings below
Lawrence_movie_ratings = [
            {'title':'Predator', 'rating':4.9},
            {'title':'Final Destination', 'rating':4.9},
            {'title':'Mission Impossible', 'rating':4},
            {'title':"Beverly Hills Cop", 'rating':3},
            {'title':'Exorcist, The', 'rating':4.8},
            {'title':'Waiting to Exhale', 'rating':3.9},
            {'title':'Avengers, The', 'rating':4.5},
            {'title':'Omen, The', 'rating':5.0}
         ] 
Lawrence_movie_ratings = pd.DataFrame(Lawrence_movie_ratings)
Lawrence_movie_ratings

,title,rating
0,Predator,4.9
1,Final Destination,4.9
2,Mission Impossible,4.0
3,Beverly Hills Cop,3.0
4,"Exorcist, The",4.8
5,Waiting to Exhale,3.9
6,"Avengers, The",4.5
7,"Omen, The",5.0


In [20]:
# Extracting movie Ids from movies_df and updating lawrence_movie_ratings with movie Ids.
movies_ids = Movies_df[Movies_df['title'].isin(Lawrence_movie_ratings['title'].tolist())]

# Merge Lawrence movie Id and ratings into the lawrence_movie_ratings data frame. 
# (This action merges both data frames by the title column)
Lawrence_ratings_with_ids = pd.merge(movies_ids,Lawrence_movie_ratings)
# Display the merged and updated data frame.
Lawrence_ratings_with_ids


,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
0,4,Waiting to Exhale,1995,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,3.9
1,1350,"Omen, The",1976,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,5.0
2,45662,"Omen, The",2006,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,5.0
3,1997,"Exorcist, The",1973,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,4.8
4,2153,"Avengers, The",1998,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.5
5,89745,"Avengers, The",2012,0,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,4.5
6,3409,Final Destination,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,4.9
7,3527,Predator,1987,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,4.9
8,4085,Beverly Hills Cop,1984,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,3.0


In [21]:
#Drop information we don't need such as year and genres
final_input_df = Lawrence_ratings_with_ids[['movieId','title','rating']]

#Final input dataframe
# (If a movie you added in above isn't here, then it might not be in the original dataframe or it might spelled differently, please check capitalisation.)
final_input_df

,movieId,title,rating
0,4,Waiting to Exhale,3.9
1,1350,"Omen, The",5.0
2,45662,"Omen, The",5.0
3,1997,"Exorcist, The",4.8
4,2153,"Avengers, The",4.5
5,89745,"Avengers, The",4.5
6,3409,Final Destination,4.9
7,3527,Predator,4.9
8,4085,Beverly Hills Cop,3.0


In [22]:
# filter the selection by outputing movies that exist in both lawrence_movie_ratings and movies_with_genres
input_watched_movies = Movies_df[Movies_df['movieId'].isin(final_input_df['movieId'].tolist())]
input_watched_movies

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
3,4,Waiting to Exhale,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1038,1350,"Omen, The",1976,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1472,1997,"Exorcist, The",1973,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1611,2153,"Avengers, The",1998,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2547,3409,Final Destination,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2636,3527,Predator,1987,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3051,4085,Beverly Hills Cop,1984,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
6216,45662,"Omen, The",2006,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
7693,89745,"Avengers, The",2012,0,1,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.

In [23]:
# First, reset index to default and drop the existing index.
input_watched_movies = input_watched_movies.reset_index(drop=True)
# Next,drop redundant columns
lawrence_genres_df = input_watched_movies.drop('movieId', 1).drop('title', 1).drop('year', 1)
# view chamges
lawrence_genres_df

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
6,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
8,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0


In [24]:
# confirm the shapes of our data frames to guide us as we do matrix multiplication
print(lawrence_genres_df.T.shape)
print(final_input_df['rating'].shape)

(20, 9)
(9,)


In [25]:
# find the dot product of transpose of Lawrence_genres_df by lawrence rating column
lawrence_profile = lawrence_genres_df.T.dot(final_input_df['rating'])

# Let's see the result
lawrence_profile.sort_values(ascending=False).head(10)

Thriller     18.9
Action       17.2
Horror       14.9
Drama        13.3
Mystery      10.0
Comedy        8.8
Adventure     7.8
Sci-Fi        7.5
Crime         4.9
Romance       3.9
dtype: float64

In [26]:
# set the index to the movieId
genre_df = Movies_df.set_index(Movies_df['movieId'])
# view the head
genre_df.head()

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,1,Toy Story,1995,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Jumanji,1995,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,Grumpier Old Men,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,4,Waiting to Exhale,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
5,5,Father of the Bride Part II,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Delete four unnecessary columns.
genre_df = genre_df.drop('movieId', 1).drop('title', 1).drop('year', 1)
# View the changes.
genre_df.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


With Lawrence's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on his profile and recommend the top k movies that match his preference.

In [28]:
# Multiply the genres by the weights and then take the weighted average.
lawrence_recommendation_df = (( genre_df * lawrence_profile ).sum(axis=1))/(lawrence_profile.sum())
# view the recommendation table
lawrence_recommendation_df.head()

movieId
1    0.150635
2    0.070780
3    0.115245
4    0.235935
5    0.079855
dtype: float64

Let's sort the recommendation table in descending order

In [29]:
# Sort values Descendingly
lawrence_recommendation_df = lawrence_recommendation_df.sort_values(ascending=False)
# view head
lawrence_recommendation_df.head()

movieId
81132    0.869328
43932    0.742287
7235     0.707804
36509    0.692377
79132    0.678766
dtype: float64

Now here's the recommendation table! Complete with movie details and genres for the top 20 movies that match Lawrence's profile.

In [30]:
# first make a copy of the original movies_df
movies_df = Movies_df[['movieId','title','year']].copy()
# Then set its index to movieId
movies_df = movies_df.set_index('movieId')
# Next enlist the top 20 recommended movieIds we defined above
lawrence_top_20_list = lawrence_recommendation_df.head(20).keys()
# finally slice these indices from the copied movies df and save in a variable
lawrence_final_recommendations = movies_df.loc[lawrence_top_20_list]
# Now display the top 20 movies in descending order of preference
lawrence_final_recommendations

,title,year
movieId,,
81132,Rubber,2010
43932,Pulse,2006
7235,Ichi the Killer (Koroshiya 1),2001
36509,"Cave, The",2005
79132,Inception,2010
51545,Pusher III: I'm the Angel of Death,2005
198,Strange Days,1995
26701,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,1989
6395,"Crazies, The (a.k.a. Code Name: Trixie)",1973


## Let's combine all together in one function

In [31]:
def get_recommended_movies(movies_df,user_movie_ratings, k_items):
    """
    Recommends movies based on user ratings and movie genres.

    Args:
        movies_df (pandas.DataFrame): DataFrame containing movie information, including movieId, title, year, and genres.
        user_movie_ratings (dict): User ratings for movies in the form of a dictionary.
                                  Key: movie title (str), Value: rating (float).
        k_items (int): Number of top recommended movies to return.

    Returns:
        pandas.DataFrame: DataFrame containing the top recommended movies with movieId, title, and year.

    """

    # transform the user ratings to a pandas dataframe.
    user_movie_ratings = pd.DataFrame(user_movie_ratings)
    
    # Extracting movie Ids from movies_df and updating lawrence_movie_ratings with movie Ids.
    movies_ids = movies_df[movies_df['title'].isin(user_movie_ratings['title'].tolist())]

    # Merge Lawrence movie Id and ratings into the lawrence_movie_ratings data frame. 
    user_ratings_with_ids = pd.merge(movies_ids,user_movie_ratings)

    # Drop information we don't need such as year and genres
    final_input_df = user_ratings_with_ids[['movieId','title','rating']]
    
    # First, reset index to default and drop the existing inde, and drop unused columns.
    user_genres_df = user_ratings_with_ids.reset_index(drop=True).drop('movieId', 1).drop('title', 1).drop('year', 1)

    # find the dot product of transpose of user_genres_df by user_ratings column to assign a weight for each genre that this user prefer.
    user_profile = user_genres_df.T.dot(final_input_df['rating'])

    # set the index to the movieId and Delete unnecessary columns.
    genre_df = movies_df.set_index(Movies_df['movieId']).drop('movieId', 1).drop('title', 1).drop('year', 1)
    
    # Multiply the genres by the  user's weights and then take the weighted average to get the recommendation table.
    user_recommendation_df = (( genre_df * user_profile ).sum(axis=1))/(user_profile.sum())
    
    # Sort values Descendingly
    user_recommendation_df = user_recommendation_df.sort_values(ascending=False)
    
    # first make a copy of the original movies_df
    movies = movies_df[['movieId','title','year']].copy()
    
    # Then set its index to movieId
    movies = movies.set_index('movieId')
    
    # Next enlist the top k recommended movieIds we defined above
    user_top_k_list = user_recommendation_df.head(k_items).keys()
    
    # finally slice these indices from the copied movies df and save in a variable that will be returned.
    user_final_recommendations = movies.loc[user_top_k_list]
    
    # Now return the top k movies in descending order of preference
    return user_final_recommendations


In [32]:
# Assuming that our user called Lawrence, so on a scale of 0 to 5, with 0 min and 5 max, see Lawrence's movie ratings below
Lawrence_movie_ratings = [
            {'title':'Predator', 'rating':4.9},
            {'title':'Final Destination', 'rating':4.9},
            {'title':'Mission Impossible', 'rating':4},
            {'title':"Beverly Hills Cop", 'rating':3},
            {'title':'Exorcist, The', 'rating':4.8},
            {'title':'Waiting to Exhale', 'rating':3.9},
            {'title':'Avengers, The', 'rating':4.5},
            {'title':'Omen, The', 'rating':5.0}
         ] 

In [33]:
rec_Lawrence_movies = get_recommended_movies(Movies_df,Lawrence_movie_ratings,10)
rec_Lawrence_movies

,title,year
movieId,,
81132,Rubber,2010
43932,Pulse,2006
36509,"Cave, The",2005
79132,Inception,2010
7235,Ichi the Killer (Koroshiya 1),2001
54771,"Invasion, The",2007
6395,"Crazies, The (a.k.a. Code Name: Trixie)",1973
74685,"Crazies, The",2010
60684,Watchmen,2009
